In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score
import joblib

In [9]:
# Set random seed for reproducibility
np.random.seed(42)
n_samples = 5000

print("🔧 Generating aircraft maintenance training data...")

# Generate base features
data = {
    "aircraft_id": [f"AC{str(i).zfill(4)}" for i in range(n_samples)],
    "aircraft_age_years": np.round(np.random.uniform(1, 25, n_samples), 1),
    "hours_since_last_maintenance": np.round(np.random.uniform(10, 500, n_samples), 1),
    "cycles_since_last_maintenance": np.random.randint(5, 300, n_samples),
    "last_maintenance_type": np.random.choice(["A-check", "B-check", "C-check", "D-check"], n_samples),
    "avg_flight_duration_last_30_days": np.round(np.random.uniform(1.5, 12, n_samples), 1),
    "total_flight_hours": np.round(np.random.uniform(5000, 80000, n_samples), 1),
    "total_cycles": np.random.randint(2000, 40000, n_samples),
    "reported_minor_issues_last_30_days": np.random.randint(0, 15, n_samples),
    "ambient_temperature_avg": np.round(np.random.uniform(-20, 45, n_samples), 1),
    "harsh_landing_count_last_30_days": np.random.randint(0, 8, n_samples),
    "manufacturer": np.random.choice(["Boeing", "Airbus", "Embraer", "Bombardier"], n_samples),
    "model_type": np.random.choice(["737-800", "747-400", "A320", "A350", "E190", "CRJ-900"], n_samples),
    "utilization_rate": np.round(np.random.uniform(0.5, 6.5, n_samples), 2)
}

df = pd.DataFrame(data)
print(f"✅ Generated {len(df)} samples")
df.head()


🔧 Generating aircraft maintenance training data...
✅ Generated 5000 samples


,aircraft_id,aircraft_age_years,hours_since_last_maintenance,cycles_since_last_maintenance,last_maintenance_type,avg_flight_duration_last_30_days,total_flight_hours,total_cycles,reported_minor_issues_last_30_days,ambient_temperature_avg,harsh_landing_count_last_30_days,manufacturer,model_type,utilization_rate
0,AC0000,10.0,202.9,187,A-check,5.8,5967.8,34614,7,-13.3,4,Airbus,737-800,5.95
1,AC0001,23.8,242.0,40,A-check,2.5,54598.3,20968,10,45.0,2,Boeing,A350,1.40
2,AC0002,18.6,428.7,41,D-check,9.8,68542.8,19985,4,-9.3,3,Airbus,737-800,1.31
3,AC0003,15.4,176.6,18,B-check,8.7,63895.5,19191,8,23.9,0,Airbus,CRJ-900,1.23
4,AC0004,4.7,436.1,127,A-check,11.5,28280.0,33908,10,41.7,4,Embraer,A350,2.66


In [10]:
# Normalize features to 0-1 scale for better control
age_norm = df["aircraft_age_years"] / 25
hours_norm = df["hours_since_last_maintenance"] / 500
cycles_norm = df["cycles_since_last_maintenance"] / 300
issues_norm = df["reported_minor_issues_last_30_days"] / 15
landing_norm = df["harsh_landing_count_last_30_days"] / 8
utilization_norm = df["utilization_rate"] / 6.5
flight_hours_norm = df["total_flight_hours"] / 80000

# Maintenance type impact (older checks = higher risk)
maintenance_risk = df["last_maintenance_type"].map({
    "A-check": 0.05,
    "B-check": 0.10,
    "C-check": 0.15,
    "D-check": 0.02  # Just had major maintenance
})

# Manufacturer reliability factor
manufacturer_risk = df["manufacturer"].map({
    "Boeing": 0.0,
    "Airbus": 0.0,
    "Embraer": 0.05,
    "Bombardier": 0.05
})

# Temperature stress (extreme temps = higher risk)
temp_risk = np.abs(df["ambient_temperature_avg"]) / 45 * 0.1

# Calculate comprehensive risk score with realistic weights
risk_score = (
    0.15 * age_norm +                    # 15% weight on aircraft age
    0.20 * hours_norm +                  # 20% weight on hours since maintenance
    0.15 * cycles_norm +                 # 15% weight on cycles
    0.25 * issues_norm +                 # 25% weight on reported issues (HIGHEST)
    0.10 * landing_norm +                # 10% weight on harsh landings
    0.05 * utilization_norm +            # 5% weight on utilization rate
    0.05 * flight_hours_norm +           # 5% weight on total hours
    maintenance_risk +                    # Maintenance type adjustment
    manufacturer_risk +                   # Manufacturer adjustment
    temp_risk                             # Temperature stress
)

# Add realistic random variation
risk_score += np.random.normal(0, 0.08, n_samples)
risk_score = np.clip(risk_score, 0, 1)

# Convert to probability with sigmoid for smooth distribution
probability = 1 / (1 + np.exp(-10 * (risk_score - 0.5)))

# Create binary target (failure risk in next 7 days)
threshold = 0.50
df["failure_probability"] = probability
df["failure_risk_binary"] = (probability > threshold).astype(int)

# Create risk levels
def assign_risk_level(prob):
    if prob < 0.25:
        return "Low"
    elif prob < 0.5:
        return "Medium"
    elif prob < 0.75:
        return "High"
    else:
        return "Critical"

df["failure_risk_level"] = df["failure_probability"].apply(assign_risk_level)

# Create recommended actions
def assign_action(prob):
    if prob < 0.25:
        return "Continue"
    elif prob < 0.5:
        return "Monitor"
    elif prob < 0.75:
        return "Schedule Maintenance"
    else:
        return "Immediate Maintenance"

df["recommended_action"] = df["failure_probability"].apply(assign_action)

print(f"\n📊 Class Distribution:")
print(df["failure_risk_binary"].value_counts())
print(f"\nPositive class rate: {df['failure_risk_binary'].mean():.2%}")
print(f"\nSample predictions range: {probability.min():.2f} to {probability.max():.2f}")

print(f"\n📊 Risk Level Distribution:")
print(df["failure_risk_level"].value_counts())

df[["aircraft_id", "failure_probability", "failure_risk_level", "recommended_action"]].head(10)


📊 Class Distribution:
1    3909
0    1091
Name: failure_risk_binary, dtype: int64

Positive class rate: 78.18%

Sample predictions range: 0.01 to 0.99

📊 Risk Level Distribution:
Critical    2553
High        1356
Medium       744
Low          347
Name: failure_risk_level, dtype: int64


,aircraft_id,failure_probability,failure_risk_level,recommended_action
0,AC0000,0.586288,High,Schedule Maintenance
1,AC0001,0.874081,Critical,Immediate Maintenance
2,AC0002,0.298516,Medium,Monitor
3,AC0003,0.645216,High,Schedule Maintenance
4,AC0004,0.730269,High,Schedule Maintenance
5,AC0005,0.541097,High,Schedule Maintenance
6,AC0006,0.681534,High,Schedule Maintenance
7,AC0007,0.771730,Critical,Immediate Maintenance
8,AC0008,0.807205,Critical,Immediate Maintenance
9,AC0009,0.949206,Critical,Immediate Maintenance


In [11]:
# Prepare features and target
X = df.drop(columns=["aircraft_id", "failure_probability", "failure_risk_binary", 
                     "failure_risk_level", "recommended_action"])
y_binary = df["failure_risk_binary"]
y_risk_level = df["failure_risk_level"]

# Define preprocessing
categorical_cols = ["last_maintenance_type", "manufacturer", "model_type"]
numerical_cols = list(X.columns.difference(categorical_cols))

print(f"Numerical features: {numerical_cols}")
print(f"Categorical features: {categorical_cols}")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
    ]
)

# Create binary classification model (for failure probability)
binary_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42
)

# Create multi-class model (for risk levels)
risk_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    class_weight='balanced'
)

# Create pipelines
binary_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", binary_model)
    ]
)

risk_pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", risk_model)
    ]
)

print("✅ Pipelines created successfully")


Numerical features: ['aircraft_age_years', 'ambient_temperature_avg', 'avg_flight_duration_last_30_days', 'cycles_since_last_maintenance', 'harsh_landing_count_last_30_days', 'hours_since_last_maintenance', 'reported_minor_issues_last_30_days', 'total_cycles', 'total_flight_hours', 'utilization_rate']
Categorical features: ['last_maintenance_type', 'manufacturer', 'model_type']
✅ Pipelines created successfully


In [12]:
# Split data with stratification
X_train, X_test, y_train_binary, y_test_binary, y_train_risk, y_test_risk = train_test_split(
    X, y_binary, y_risk_level, test_size=0.2, random_state=42, stratify=y_binary
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Train binary model
print("\n🚀 Training Binary Classification Model (Failure Probability)...")
binary_pipeline.fit(X_train, y_train_binary)
print("✅ Training complete!")

# Make predictions
y_prob_binary = binary_pipeline.predict_proba(X_test)[:, 1]
y_pred_binary = binary_pipeline.predict(X_test)

# Evaluate performance
print("\n" + "="*60)
print("📈 BINARY MODEL PERFORMANCE (Failure Probability)")
print("="*60)
print(f"\nROC-AUC Score: {roc_auc_score(y_test_binary, y_prob_binary):.4f}")
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_binary):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test_binary, y_pred_binary, 
                          target_names=['No Failure Risk', 'Failure Risk']))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test_binary, y_pred_binary))

# Check prediction distribution
print("\n📊 Prediction Probability Distribution on Test Set:")
prob_bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
prob_labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
prob_dist = pd.cut(y_prob_binary, bins=prob_bins, labels=prob_labels)
print(prob_dist.value_counts().sort_index())

Training set: 4000 samples
Test set: 1000 samples

🚀 Training Binary Classification Model (Failure Probability)...
✅ Training complete!

📈 BINARY MODEL PERFORMANCE (Failure Probability)

ROC-AUC Score: 0.8877
Accuracy: 0.8540

📊 Classification Report:
                 precision    recall  f1-score   support

No Failure Risk       0.72      0.53      0.61       218
   Failure Risk       0.88      0.94      0.91       782

       accuracy                           0.85      1000
      macro avg       0.80      0.74      0.76      1000
   weighted avg       0.85      0.85      0.85      1000


📊 Confusion Matrix:
[[116 102]
 [ 44 738]]

📊 Prediction Probability Distribution on Test Set:
0-20%       94
20-40%      48
40-60%      43
60-80%      70
80-100%    745
dtype: int64


In [13]:
print("\n🚀 Training Multi-Class Model (Risk Levels)...")
risk_pipeline.fit(X_train, y_train_risk)
print("✅ Training complete!")

# Make predictions
y_pred_risk = risk_pipeline.predict(X_test)
y_prob_risk = risk_pipeline.predict_proba(X_test)

# Evaluate performance
print("\n" + "="*60)
print("📈 RISK LEVEL MODEL PERFORMANCE")
print("="*60)
print(f"\nAccuracy: {accuracy_score(y_test_risk, y_pred_risk):.4f}")

print("\n📊 Classification Report:")
print(classification_report(y_test_risk, y_pred_risk))

print("\n📊 Confusion Matrix:")
cm = confusion_matrix(y_test_risk, y_pred_risk, labels=['Low', 'Medium', 'High', 'Critical'])
cm_df = pd.DataFrame(cm, 
                     index=['Low', 'Medium', 'High', 'Critical'],
                     columns=['Low', 'Medium', 'High', 'Critical'])
print(cm_df)



🚀 Training Multi-Class Model (Risk Levels)...
✅ Training complete!

📈 RISK LEVEL MODEL PERFORMANCE

Accuracy: 0.6120

📊 Classification Report:
              precision    recall  f1-score   support

    Critical       0.75      0.83      0.79       494
        High       0.47      0.41      0.44       288
         Low       0.43      0.44      0.44        61
      Medium       0.40      0.35      0.37       157

    accuracy                           0.61      1000
   macro avg       0.51      0.51      0.51      1000
weighted avg       0.60      0.61      0.60      1000


📊 Confusion Matrix:
          Low  Medium  High  Critical
Low        27      27     7         0
Medium     27      55    58        17
High        8      44   119       117
Critical    1      13    69       411


In [14]:
print("\n🧪 TESTING WITH SAMPLE SCENARIOS")
print("="*60)

test_cases = [
    {
        "name": "✅ Low Risk Aircraft (New, Well-Maintained)",
        "data": {
            'aircraft_age_years': 3.5,
            'hours_since_last_maintenance': 45.0,
            'cycles_since_last_maintenance': 25,
            'last_maintenance_type': 'A-check',
            'avg_flight_duration_last_30_days': 3.5,
            'total_flight_hours': 8000.0,
            'total_cycles': 4000,
            'reported_minor_issues_last_30_days': 0,
            'ambient_temperature_avg': 22.0,
            'harsh_landing_count_last_30_days': 0,
            'manufacturer': 'Boeing',
            'model_type': '737-800',
            'utilization_rate': 2.5
        }
    },
    {
        "name": "⚠️ Medium Risk Aircraft (Moderate Age, Some Issues)",
        "data": {
            'aircraft_age_years': 12.0,
            'hours_since_last_maintenance': 250.0,
            'cycles_since_last_maintenance': 150,
            'last_maintenance_type': 'B-check',
            'avg_flight_duration_last_30_days': 5.5,
            'total_flight_hours': 35000.0,
            'total_cycles': 18000,
            'reported_minor_issues_last_30_days': 5,
            'ambient_temperature_avg': 30.0,
            'harsh_landing_count_last_30_days': 2,
            'manufacturer': 'Airbus',
            'model_type': 'A320',
            'utilization_rate': 4.2
        }
    },
    {
        "name": "🔴 High Risk Aircraft (Old, Multiple Issues)",
        "data": {
            'aircraft_age_years': 22.0,
            'hours_since_last_maintenance': 480.0,
            'cycles_since_last_maintenance': 290,
            'last_maintenance_type': 'A-check',
            'avg_flight_duration_last_30_days': 8.5,
            'total_flight_hours': 72000.0,
            'total_cycles': 36000,
            'reported_minor_issues_last_30_days': 13,
            'ambient_temperature_avg': 42.0,
            'harsh_landing_count_last_30_days': 7,
            'manufacturer': 'Embraer',
            'model_type': 'E190',
            'utilization_rate': 6.0
        }
    },
    {
        "name": "🟡 Critical Edge Case (Recent Major Maintenance, but Old)",
        "data": {
            'aircraft_age_years': 20.0,
            'hours_since_last_maintenance': 25.0,
            'cycles_since_last_maintenance': 15,
            'last_maintenance_type': 'D-check',
            'avg_flight_duration_last_30_days': 4.0,
            'total_flight_hours': 65000.0,
            'total_cycles': 32000,
            'reported_minor_issues_last_30_days': 1,
            'ambient_temperature_avg': 18.0,
            'harsh_landing_count_last_30_days': 0,
            'manufacturer': 'Boeing',
            'model_type': '747-400',
            'utilization_rate': 3.5
        }
    }
]

for test_case in test_cases:
    test_df = pd.DataFrame([test_case['data']])
    
    # Binary prediction
    prob_binary = binary_pipeline.predict_proba(test_df)[0][1] * 100
    pred_binary = "At Risk" if prob_binary > 50 else "Normal"
    
    # Risk level prediction
    risk_level = risk_pipeline.predict(test_df)[0]
    risk_probs = risk_pipeline.predict_proba(test_df)[0]
    risk_confidence = max(risk_probs) * 100
    
    # Recommended action
    if prob_binary < 25:
        action = "Continue"
    elif prob_binary < 50:
        action = "Monitor"
    elif prob_binary < 75:
        action = "Schedule Maintenance"
    else:
        action = "Immediate Maintenance"
    
    print(f"\n{test_case['name']}")
    print(f"  Failure Probability: {prob_binary:.1f}% ({pred_binary})")
    print(f"  Risk Level: {risk_level} (confidence: {risk_confidence:.1f}%)")
    print(f"  Recommended Action: {action}")


🧪 TESTING WITH SAMPLE SCENARIOS

✅ Low Risk Aircraft (New, Well-Maintained)
  Failure Probability: 0.1% (Normal)
  Risk Level: Low (confidence: 73.1%)
  Recommended Action: Continue

⚠️ Medium Risk Aircraft (Moderate Age, Some Issues)
  Failure Probability: 94.4% (At Risk)
  Risk Level: High (confidence: 35.0%)
  Recommended Action: Immediate Maintenance

🔴 High Risk Aircraft (Old, Multiple Issues)
  Failure Probability: 100.0% (At Risk)
  Risk Level: Critical (confidence: 92.3%)
  Recommended Action: Immediate Maintenance

🟡 Critical Edge Case (Recent Major Maintenance, but Old)
  Failure Probability: 0.2% (Normal)
  Risk Level: Low (confidence: 47.7%)
  Recommended Action: Continue


In [15]:
# Save the trained models
binary_model_path = "equipment_failure_prob_model.pkl"
risk_model_path = "equipment_failure_risk_model.pkl"

joblib.dump(binary_pipeline, binary_model_path)
joblib.dump(risk_pipeline, risk_model_path)

# Save metadata
metadata = {
    "model_version": "1.0",
    "training_samples": len(X_train),
    "test_samples": len(X_test),
    "features": list(X.columns),
    "categorical_features": categorical_cols,
    "numerical_features": numerical_cols,
    "risk_levels": ["Low", "Medium", "High", "Critical"],
    "actions": ["Continue", "Monitor", "Schedule Maintenance", "Immediate Maintenance"],
    "binary_model_accuracy": float(accuracy_score(y_test_binary, y_pred_binary)),
    "binary_model_auc": float(roc_auc_score(y_test_binary, y_prob_binary)),
    "risk_model_accuracy": float(accuracy_score(y_test_risk, y_pred_risk))
}

import json
with open("model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*60)
print("✅ MODELS SAVED SUCCESSFULLY!")
print("="*60)
print(f"\nFiles created:")
print(f"1. {binary_model_path}")
print(f"2. {risk_model_path}")
print(f"3. model_metadata.json")


✅ MODELS SAVED SUCCESSFULLY!

Files created:
1. equipment_failure_prob_model.pkl
2. equipment_failure_risk_model.pkl
3. model_metadata.json
